# Работа с реляционными БД

## Термины

* **БД** — база данных, то, где хранятся данные (вот так)
* **СУБД** — система управления базами данных — программное обеспечение для работы в с данными
* **Отношение** термин [*реляционной алгебры*](https://ru.wikipedia.org/wiki/%D0%A0%D0%B5%D0%BB%D1%8F%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D0%B0%D0%BB%D0%B3%D0%B5%D0%B1%D1%80%D0%B0) — аналог таблицы истинности логического отношения
* **Таблица** — набор данных в БД, соответствующий отношению в реляционной алгебре

## Структура и нормализация данных

1. 1NF — отношение состоит из кортежей одинаковой природы, элементы кортежей атомарны
2. 2NF — каждый неключевой атрибут функционально зависит от ключа
3. 3NF — нет транзитивных функциональных зависимостей

...

См. Дейт К. Дж. Введение в системы баз данных

## Поехали уже =)

![Bobby Tables](https://imgs.xkcd.com/comics/exploits_of_a_mom.png)

In [3]:
import sqlite3
import sqlalchemy

conn = sqlite3.connect("students.db")  # Как оригинально!
cursor = conn.cursor()

А дальше соединения с любой реляционной БД и (для любых типов реляционных СУБД) должны (желательно) удовлетворять [Python DB-API 2.0](https://www.python.org/dev/peps/pep-0249/).

In [4]:
with open("11.drop_ddl.sql", 'r', encoding='utf-8') as f: drop_ddl = f.read()
with open("11.create_ddl.sql", 'r', encoding='utf-8') as f: create_ddl = f.read()
with open("11.insert_dml.sql", 'r', encoding='utf-8') as f: insert_dml = f.read()

In [5]:
if True:
    for st in drop_ddl.split(';'):
        cursor.execute(st)
    conn.commit()

In [6]:
for st in create_ddl.split(';'):
    cursor.execute(st)
conn.commit()

In [7]:
for st in insert_dml.split(';'):
    cursor.execute(st)
conn.commit()

In [8]:
conn.close()

Теперь попробуем немного [ORM](https://en.wikipedia.org/wiki/Object-relational_mapping) — [SQLAlchemy](https://ru.wikibooks.org/wiki/SQLAlchemy)

In [9]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.ext.declarative import declarative_base

DeclBase = declarative_base()

In [10]:
engine = sqlalchemy.create_engine('sqlite:///students.db', echo=False)  # echo=True для логгинга
Session = sessionmaker(bind=engine)
session = Session()

In [11]:
class Program(DeclBase):
    __tablename__ = 'programs'
    id = Column(Integer, primary_key=True)
    name = Column(String)

    students = relationship("Student", backref="program")
    programs_courses = relationship("Program_course", backref="p_c")

    def __init__(self, name):
        self.name = name
        
class Course(DeclBase):
    __tablename__ = 'courses'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    
    marks = relationship("Mark", backref="marks")
    programs_courses = relationship("Program_course", backref="p_c_second")

    def __init__(self, name):
        self.name = name
        
class Program_course(DeclBase):
    __tablename__ = 'programs_courses'
    semester_number = Column(Integer, primary_key=True)
    program_id = Column(Integer, ForeignKey('programs.id'), primary_key=True)
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)

    def __init__(self, semester_number, program_id, course_id):
        self.semester_number = semester_number
        self.program_id = program_id
        self.course_id = course_id
        
class Mark(DeclBase):
    __tablename__ = 'marks'
    student_id = Column(Integer, ForeignKey('students.id'), primary_key=True)
    course_id = Column(Integer, ForeignKey('courses.id'), primary_key=True)
    mark = Column(Integer)
    
    students = relationship("Student", backref="mark")  

    def __init__(self, student_id, course_id, mark):
        self.mark = mark
        self.student_id = student_id
        self.course_id = course_id
        

class Student(DeclBase):
    __tablename__ = 'students'
    id = Column(Integer, primary_key=True)
    card = Column(String)
    surname = Column(String)
    name = Column(String)
    patronymic = Column(String)
    
    program_id = Column(Integer, ForeignKey('programs.id'))
    marks = relationship("Mark", backref="student")
    
    def __init__(self, card, surname, name, patronymic, program):
        self.card = card
        self.surname = surname
        self.name = name
        self.patronymic = patronymic
        self.program = program

In [12]:
se = Program("Программная инженерия")

st1 = Student("002002", "Иванов", "Пётр", "Сидорович", se)
st2 = Student("002003", "Петрова", "Исидора", "Ивановна", se)

session.add_all([se, st1, st2])
session.commit()

In [13]:
import sys

print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name)

Программы и студенты
Программа:  математика и информационные технологии
-  Илья
Программа:  теоретическая физика
-  Данила
-  Екатерина
Программа:  Программная инженерия
-  Пётр
-  Исидора


In [14]:
for s in session.query(Student).filter_by(surname="Аргонова"):
    s.program = se
session.commit()

print("Программы и студенты")
for p in session.query(Program):
    print("Программа: ", p.name)
    for s in p.students:
        print("- ", s.name)

Программы и студенты
Программа:  математика и информационные технологии
-  Илья
Программа:  теоретическая физика
-  Данила
-  Екатерина
Программа:  Программная инженерия
-  Пётр
-  Исидора


In [22]:
print("Успеваемость по предметам")
for c in session.query(Course):
    print("Предмет: ", c.name)
    for m in c.marks:
        s_id = session.query(Student).filter_by(id = m.student_id)[0]
        print("- ", m.mark, s_id.name, s_id.surname)

Успеваемость по предметам
Предмет:  Алгебра
-  2 Данила Скворцов
-  3 Екатерина Сорокина
Предмет:  Дискретная математика
-  3 Илья Нагай
Предмет:  Математический анализ
-  5 Илья Нагай
-  5 Екатерина Сорокина
Предмет:  Физика
-  3 Илья Нагай
-  5 Данила Скворцов
-  4 Екатерина Сорокина
Предмет:  Физическая культура
-  4 Илья Нагай
-  4 Данила Скворцов
-  5 Екатерина Сорокина


In [29]:
print("Расписание программ")
for p in session.query(Program):
    print("Программа: ", p.name)
    for pc in session.query(Program_course).filter_by(program_id=p.id):
        print("- ", session.query(Course).filter_by(id=pc.course_id)[0].name)

Расписание программ
Программа:  математика и информационные технологии
-  Дискретная математика
-  Математический анализ
-  Физика
-  Физическая культура
Программа:  теоретическая физика
-  Алгебра
-  Математический анализ
-  Физика
-  Физическая культура
Программа:  Программная инженерия
